In [1]:
from tensorflow.keras.datasets import cifar10

(x_full, y_full), (x_test, y_test) = cifar10.load_data()

x_full = x_full.astype('float32') / 255.0  
x_test = x_test.astype('float32') / 255.0

# Calculate split sizes
num_train = int(0.6 * len(x_full)) 
num_val = int(0.2 * len(x_full)) 

# Split into training and validation sets
training_x = x_full[:num_train]
training_y = y_full[:num_train]
val_x = x_full[num_train: (num_train + num_val)]
val_y = y_full[num_train: (num_train + num_val)]
test_x = x_full[(num_train + num_val):]
test_y = y_full[(num_train + num_val):]




170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 36s 0us/step
